# Quality of Experience (QoE) Script - for ARM architecture

**Overview**

This script is designed for automated collection of Quality of Experience (QoE) for YouTube videos.

**Main components**
- **`generate_script()`**: Generates a JavaScript for reporting QoE metrics to a specified server. The script includes functions for posting data to the server periodically and handling state and playback quality changes in a video player.
- **`class DebugWatchYouTubeVideoTask`**: A task that opens a YouTube video in a browser, watches it for a specified duration, and uses the generated script to report various QoE metrics like playback quality and state changes to collection server periodically.
- **`generateRunServerFile()`**: Generates the Python script required to run a uvicorn server, which will host a specific ASGI application. The application is specified to handle QoE data collection, configured to use HTTPs for secure communication.
- **`class StartQoECollectionServerTask`**: A task that automate the setup and launch of a QoE collection server by managing essential steps such as freeing the required port, generating SSL certificates, creating a data directory, setting environment variables, and running the server. It effectively handles initial checks and configurations to ensure the server starts smoothly and securely for optimal QoE data collection. This class significantly streamlines the process, making it robust and error-resilient, facilitating reliable data collection in a networked environment.
- **`class uploadAllFilesTask`**:  This task iterates over files in a specified directory, uploads them to a designated Google Cloud Storage bucket, and reports the upload status.

In [ ]:
import os
import time

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline, Task

from returns.pipeline import is_successful
from returns.result import Result, Success, Failure

from typing import List, Optional
import random
import subprocess
import socket
import signal
import json
import datetime

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from google.cloud import storage

Most of the other code is not different from previous netunicorn examples. Let's use correct credentials for our infrastructure (in your case they could be different, if you don't have netunicorn instance in your organization - you can deploy your own locally for testing purposes, see https://netunicorn.cs.ucsb.edu/examples for details)

In [ ]:
# if you have .env file locally for storing credentials, skip otherwise
if '.env' in os.listdir():
    from dotenv import load_dotenv
    load_dotenv(".env")

NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'http://localhost:26611')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', 'test')
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', 'test')
print(NETUNICORN_ENDPOINT, NETUNICORN_LOGIN, NETUNICORN_PASSWORD)

client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
client.healthcheck()

Let's get some nodes for execution. As usual, for demonstration purposes of this notebook we will take our Raspberry Pi nodes, but if your infrastructure is different - feel free to modify the next cell.

In [ ]:
nodes = client.get_nodes()

# switch for showing our infrastructure vs you doing it locally on other nodes
if os.environ.get('NETUNICORN_ENDPOINT', 'http://localhost:26611') != 'http://localhost:26611':
    working_nodes = nodes.filter(lambda node: node.name.startswith("raspi")).take(1)
else:
    working_nodes = nodes.take(1)

working_nodes

In [ ]:
# generate the javascript code for the experiment
def generate_script(server_address, server_port, report_time):
    return f"""
"use strict";
/*jshint esversion: 9 */
/* jshint -W097 */

// default url where report server is located
const quality_change_url = "https://{server_address}:{server_port}/quality";
const state_change_url = "https://{server_address}:{server_port}/state";
const stats_url = "https://{server_address}:{server_port}/report";
const report_time = {report_time};

function postReport(url, jsonData) {{
    console.debug("Posting report to", url); // Logging URL
    console.debug("Data:", jsonData); //
    // this function sends json data to report server
    let xhr = new XMLHttpRequest();
    xhr.open("POST", url, true);
    xhr.setRequestHeader("Content-Type", "application/json; charset=utf-8");
    xhr.send(JSON.stringify(jsonData));
}}

function onStateChange(event) {{
    console.debug("State changed to:", event);
    // this function catch player state changes and report them
    postReport(
        state_change_url,
        {{
            video_id_and_cpn: player.getStatsForNerds().video_id_and_cpn,
            fraction: player.getVideoLoadedFraction(),
            current_time: player.getCurrentTime(),
            new_state: event,
        }}
    );
}}

function onPlaybackQualityChange(event) {{
    console.debug("Playback quality changed to:", event);
    // this function post quality changes
    postReport(
        quality_change_url,
        {{
            video_id_and_cpn: player.getStatsForNerds().video_id_and_cpn,
            fraction: player.getVideoLoadedFraction(),
            current_time: player.getCurrentTime(),
            new_quality: event,
        }}
    );
}}

function sendStats() {{
    console.debug("Sending stats...");
    // this function is executed every X ms and reports current statistics
    let stats_for_nerds = player.getStatsForNerds();
    stats_for_nerds.playback_fraction = player.getVideoLoadedFraction();
    stats_for_nerds.current_time = player.getCurrentTime();
    console.debug("Current stats:", stats_for_nerds); // Log current stats

    postReport(stats_url, stats_for_nerds);
}}

// wait until player is ready
while (!document.getElementById("movie_player")) {{
    (async () => {{
            console.debug("Waiting for player to be ready...");

        await new Promise(r => setTimeout(r, 100));
    }})();
}}

// get the player
let player = document.getElementById("movie_player");

console.log("Player found:", player); // Log player element found

// register callbacks on state and quality changes
player.addEventListener("onStateChange", onStateChange);
player.addEventListener("onPlaybackQualityChange", onPlaybackQualityChange);

// report stats for nerds every X ms
setInterval(sendStats, report_time);
"""

In [ ]:
# Class for begining youtube video watching
class DebugWatchYouTubeVideoTask(Task):
    def __init__(
        self, video_url, duration, quality, qoe_server_address, qoe_server_port, report_time
    ):
        super().__init__()
        self.video_url = video_url
        self.duration = duration
        self.quality = quality
        self.qoe_server_address = qoe_server_address
        self.qoe_server_port = qoe_server_port
        self.report_time = report_time

    def run(self):
        try:
            qoe_extension_path = os.path.join(".", "extensions", "qoe_extension")

            script_js_path = os.path.join(qoe_extension_path, "script.js")
            with open(script_js_path, "w") as f:
                f.write(generate_script(self.qoe_server_address, self.qoe_server_port, self.report_time))

            result = self.debug_watch(self.video_url, self.duration, self.quality, qoe_extension_path)

            return result
        except Exception as e:
            print(f"Error occurred: {e}")
            return Failure(f"Error occurred: {e}")
        
    def extract_qualities(text: str) -> List[int]:
        # Because of how youtube quality menu created
        lines = text.split("\n")[1:-1]

        nums = [int(s[: s.find("p")]) for s in lines]
        return nums


    def find_closest(self, options: List[int], goal: int) -> int:
        sorted_options = sorted(options)
        if not sorted_options:
            raise Exception("Youtube parsing error: quality menu block is empty")

        opt = 0
        for ind, opt in enumerate(sorted_options):
            if opt >= goal:
                if ind > 0 and (goal - sorted_options[ind - 1] < opt - goal):
                    return options.index(sorted_options[ind - 1])
                else:
                    return options.index(opt)
        return options.index(opt)
        
    def select_quality(self, driver: webdriver.Chrome, quality: int) -> None:
        settings = driver.find_element(
            By.CLASS_NAME, "ytp-settings-button"
        )  # .find_elements_by_class_name("ytp-settings-button")
        settings.click()
        menu = driver.find_elements(By.CLASS_NAME, "ytp-menuitem")
        menu[3].click()
        quality_menu = driver.find_element(By.CLASS_NAME, "ytp-quality-menu")
        options = self.extract_qualities(quality_menu.text)
        index_to_select = (
            self.find_closest(options, quality) + 1
        )  # Because we cutted first "go back to menu" option
        menu = driver.find_elements(By.CLASS_NAME, "ytp-menuitem")
        menu[index_to_select].click()
    
    def debug_watch(self, url: str, duration: Optional[int] = 100, quality: Optional[int] = None, Statsfornerds_Path: str = None) -> Result[str, str]:

        # Display size is random popular screen size
        display_number = random.randint(100, 500)
        xvfb_process = subprocess.Popen(
            ["Xvfb", f":{display_number}", "-screen", "0", "1920x1080x24"]
        )
        os.environ["DISPLAY"] = f":{display_number}"

        options = Options()
        
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-gpu")

        options.add_argument("--disable-dev-shm-usage")
        options.add_argument(f"--load-extension={Statsfornerds_Path}")
        
        options.add_argument("--disable-web-security")
        options.add_argument("--allow-running-insecure-content")
        options.add_argument("--ignore-certificate-errors")

        # logPath = os.path.join(Statsfornerds_Path, 'logfile.log')
        # with open(logPath, "w") as f:
        #     pass
        # if not os.path.exists(logPath):
        #     print("Logpath not exists")
        # else:
        #     print(f"Logpath: {logPath}")
        # options.add_argument("--enable-logging")
        # options.add_argument("--v=3") 
        # options.add_argument(f"--log-path={logPath}")

        # or press space part
        options.add_argument("--autoplay-policy=no-user-gesture-required")

        #raspi is 7 hours ahead
        desired_time = datetime.datetime(2024, 6, 13, 18, 20, 0)
        print(f"Desired time: {desired_time}")
        current_time = datetime.datetime.now()
        print(f"Current time: {current_time}")
        time_difference = desired_time - current_time
        print(f"Time difference: {time_difference}")
        wait_time_seconds = time_difference.total_seconds()
        if wait_time_seconds > 0:
            time.sleep(wait_time_seconds)

        driver = webdriver.Chrome(service=Service(), options=options)

        time.sleep(1)
        driver.get(url)
        # To make sure we stay on our page (make sure your ad-block extension does
        # not load itself as 0 page)
        # The problem is that we do not know when the adblock page will be opened,
        # so we have to make sure that we done our best to swithced to right window
        # and give youtube ~5 secs to load in bad cases

        pages = driver.window_handles

        i = 0
        driver.switch_to.window(pages[i])

        # If current is "not ours", we know that there is ours, so let's search
        while "youtube" not in driver.current_url and i < len(pages):
            i += 1
            driver.switch_to.window(pages[i])

        # For bad internet connection case - wait and retry 5 sec
        for s in range(5):
            try:
                driver.switch_to.window(pages[i])
                video = driver.find_element(By.ID, "movie_player")
                break
            except NoSuchElementException:
                time.sleep(1)
        else:
            driver.switch_to.window(pages[i])
            video = driver.find_element(By.ID, "movie_player")

        if not (quality is None):
            self.select_quality(driver, quality)

        # video.send_keys(Keys.SPACE)  # hits space for start if option not availible

        if duration is None:
            player_status = 1  # Suppose video playing now
            while player_status != 0:  # While not stopped - see docs
                time.sleep(2)  # Random 2s constant not to check to freq
                player_status = driver.execute_script(
                    "return document.getElementById('movie_player').getPlayerState()"
                )
            how = "End of video"
        else:
            time.sleep(duration)
            # video.send_keys(Keys.SPACE)
            how = "Time limit"

        # print("******Getting logs")
        # logs = driver.get_log('browser')
        # for log in logs:
        #     print(log)

        driver.close()
        xvfb_process.kill()

        return Success(how)

In [ ]:
def generateRunServerFile():
    return """
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config("netunicorn.library.tasks.qoe_youtube.qoe_collector:app", host="0.0.0.0", port=34543, ssl_keyfile="mykey.pem", ssl_certfile="mycert.pem", log_level="warning")
    server = uvicorn.Server(config)
    server.run()
"""

In [ ]:
class StartQoECollectionServerTask(Task):
    def __init__(self, data_folder):
        super().__init__()
        self.data_folder = data_folder


    def run(self):
        try:
            # Check and free the port if necessary
            port = 34543
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                if s.connect_ex(('localhost', port)) == 0:
                    print(f"Port {port} is in use. Attempting to kill...")
                    kill_command = f"lsof -t -i:{port} -sTCP:LISTEN"
                    process = subprocess.run(kill_command, shell=True, capture_output=True, text=True)
                    if process.stdout:
                        pid = process.stdout.strip()
                        os.kill(int(pid), signal.SIGKILL)
                        print(f"Killed process {pid} on port {port}")
                    else:
                        print("No process to kill.")

            # Write content to run_server.py file
            with open("run_server.py", "w") as f:
                f.write(generateRunServerFile())

            # Generate key and cert from openssl
            command = """
openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout mykey.pem -out mycert.pem -subj "/C=US/ST=California/L=Mountain View/O=UCSB/OU=IT Department/CN=www.example.com"
"""
            subprocess.run(command, shell=True)

            # Create a folder for QoE data
            os.makedirs(self.data_folder, exist_ok=True)

            # set export QOE_DATA_FOLDER
            os.environ["QOE_DATA_FOLDER"] = self.data_folder

            # Run the server
            subprocess.Popen(["python", "run_server.py"])

            return Success("Server started")

        except Exception as e:
            print(f"Error occurred: {e}")
            return Failure(f"Error occurred: {e}")

In [ ]:
# Put your google cloud storage credentials here
credentials = {
  "type": "xxx",
  "project_id": "xxx",
  "private_key_id": "xxx",
  "private_key": "xxx",
  "client_email": "xxx",
  "client_id": "xxx",
  "auth_uri": "xxx",
  "token_uri": "xxx",
  "auth_provider_x509_cert_url": "xxx",
  "client_x509_cert_url": "xxx",
  "universe_domain": "xxx"
}

In [ ]:
class uploadAllFilesTask(Task):
    def __init__(self, data_folder):
        super().__init__()
        self.data_folder = data_folder

    
    def upload_blob(self, bucket_name, source_file_name, destination_blob_name):
        # Create a client
        with open("credential.json", "w") as f:
            json.dump(credentials, f)
        storage_client = storage.Client.from_service_account_json('./credential.json')
        
        # Get the bucket
        bucket = storage_client.bucket(bucket_name)
        print(f"Bucket {bucket.name} found.")
        # Create a blob object
        blob = bucket.blob(destination_blob_name)
        print(f"Blob {blob.name} created.")
        # Upload the file
        blob.upload_from_filename(source_file_name)

        print(f"File {source_file_name} uploaded to {destination_blob_name}.")

    def run(self):
        try:
            print(f"Uploading all files from {self.data_folder}...")
            for filename in os.listdir(self.data_folder):
                filepath = os.path.join(self.data_folder, filename)
                print(filepath)
                if os.path.isfile(filepath):
                    self.upload_blob("qoe_data", filepath, filename)
            return Success("Successfully uploaded all files")
        except Exception as e:
            print(f"Error occurred: {e}")
            return Failure(f"Error occurred: {e}")

In [ ]:
# Define the url of the video
YouTube_URL = "https://www.youtube.com/watch?v=r0u5URS3VXE"

As the next step, let's assemble our pipeline.

Important notes:
- data_foler specifies the folder path at the running node.
- duration is the total watching time(in seconds).
- report_time is the report period(in milliseconds).
- qoe_server_address typically set to "0.0.0.0" because we want to run the collection server on node itself.

In [ ]:
pipeline = (
    Pipeline()
    .then(StartQoECollectionServerTask(data_folder="/qoe_data"))
    .then(DebugWatchYouTubeVideoTask(video_url = YouTube_URL, duration=26, report_time=5000, quality=None, qoe_server_address="0.0.0.0", qoe_server_port=34543))
    .then(uploadAllFilesTask(data_folder="/qoe_data"))
)

In [ ]:
# Creating the experiment
experiment = Experiment().map(pipeline, working_nodes)
experiment

We use a predefined Docker image. This image was created by taking a base `netunicorn/chromium:latest` image and installing all the required packages.

In [ ]:
from netunicorn.base import DockerImage

for deployment in experiment:
    deployment.environment_definition = DockerImage(image='sakura61777/netunicorn_local_arm:v13')

In [ ]:
experiment_label = "qoe_collection_ARM"

In [ ]:
try:
    client.delete_experiment(experiment_label)
except RemoteClientException:
    pass

client.prepare_experiment(experiment, experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status == ExperimentStatus.READY:
        break
    time.sleep(2)

Verifying that everything is correct:

In [ ]:
for deployment in client.get_experiment_status(experiment_label).experiment:
    print(f"Prepared: {deployment.prepared}, error: {deployment.error}")

And starting the execution:

In [ ]:
client.start_execution(experiment_label)

while True:
    info = client.get_experiment_status(experiment_label)
    print(info.status)
    if info.status != ExperimentStatus.RUNNING:
        break
    time.sleep(2)

In [ ]:
for report in info.execution_result:
    print(f"Node name: {report.node.name}")
    print(f"Error: {report.error}")

    result, log = report.result  # report stores results of execution and corresponding log
    
    # result is a returns.result.Result object, could be Success of Failure
    # or None is error occured during execution
    
    print(f"Result is: {type(result)}")
    print(f"Result: {result}")
    
    if isinstance(result, Result):
        data = result.unwrap() if is_successful(result) else result.failure()
        for key, value in data.items():
            print(f"{key}: {value}")

    # we also can explore logs
    for line in log:
        print(line.strip())
    print()